# 1.Importing Required Libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image

# 2.Downloading Dataset to Cloud Platform

In [ ]:
!unzip "/content/CovidDataset.zip"

Archive:  /content/CovidDataset.zip
   creating: CovidDataset/
   creating: CovidDataset/InputTestData/
   creating: CovidDataset/InputTestData/Covid/
 extracting: CovidDataset/InputTestData/Covid/COVID-112.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-113.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-114.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2537.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2538.png  
  inflating: CovidDataset/InputTestData/Covid/COVID-2539.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2540.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2541.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2542.png  
 extracting: CovidDataset/InputTestData/Covid/COVID-2543.png  
   creating: CovidDataset/InputTestData/Normal/
 extracting: CovidDataset/InputTestData/Normal/Normal-26.png  
 extracting: CovidDataset/InputTestData/Normal/Normal-27.png  
 extracting: CovidDataset/InputTestData/Normal/N

# 3.Splitting the Dataset

### Generation of training data using ImageDataGenerator with Image Augmentation

In [ ]:
train_datagen = image.ImageDataGenerator(  rescale= 1/255,  horizontal_flip= True, zoom_range= 0.2, shear_range= 0.2)

train = train_datagen.flow_from_directory(directory= '/content/CovidDataset/Train' , target_size=(224,224), batch_size= 16, class_mode= 'binary')

Found 200 images belonging to 2 classes.


In [ ]:
train.class_indices

{'Covid': 0, 'Normal': 1}

### Generation of testing data using ImageDataGenerator with Image Augmentation

In [ ]:
test_datagen = image.ImageDataGenerator(  rescale= 1/255)

test = test_datagen.flow_from_directory(directory= '/content/CovidDataset/Val' , target_size=(224,224), batch_size= 16, class_mode= 'binary')

Found 40 images belonging to 2 classes.


In [ ]:
test.class_indices

{'Covid': 0, 'Normal': 1}

# 4.Defining the Machine Learning Model

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= (224, 224, 3) ))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units= 64, activation= 'relu'))
model.add(Dropout(rate= 0.50))

model.add(Dense(units= 1, activation= 'sigmoid'))

model.compile(loss= keras.losses.binary_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

# 5.Training The Model

In [ ]:
model.fit_generator(train, steps_per_epoch= 8, epochs=10, validation_steps= 2, validation_data= test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 48s 356ms/step - loss: 0.8305 - accuracy: 0.5539 - val_loss: 0.6931 - val_accuracy: 0.5312
Epoch 2/10
8/8 [==============================] - 3s 392ms/step - loss: 0.6933 - accuracy: 0.5368 - val_loss: 0.6929 - val_accuracy: 0.5312
Epoch 3/10
8/8 [==============================] - 2s 293ms/step - loss: 0.6825 - accuracy: 0.6186 - val_loss: 0.6934 - val_accuracy: 0.3750
Epoch 4/10
8/8 [==============================] - 2s 292ms/step - loss: 0.6881 - accuracy: 0.5397 - val_loss: 0.6882 - val_accuracy: 0.5625
Epoch 5/10
8/8 [==============================] - 2s 292ms/step - loss: 0.6660 - accuracy: 0.6519 - val_loss: 0.6886 - val_accuracy: 0.5312
Epoch 6/10
8/8 [==============================] - 2s 279ms/step - loss: 0.6790 - accuracy: 0.6157 - val_loss: 0.6899 - val_accuracy: 0.3750
Epoch 7/10
8/8 [==============================] - 2s 278ms/step - loss: 0.6604 - accuracy: 0.6144 - val_loss: 0.6954 - val_accuracy: 0.6250
Epoch 8/10
8/8 [===

# 6.Evaluation of the Trained Model

In [ ]:
input_path = '/content/CovidDataset/InputTestData/Covid/COVID-112.png'

img = image.load_img(input_path, target_size= (224, 224))

img = image.img_to_array(img)/255
img = np.array([img])
img.shape

(1, 224, 224, 3)

In [ ]:
#model.predict_classes(img)
np.argmax(model.predict(img), axis=-1)

array([0])

In [ ]:
(model.predict(img) > 0.5).astype("int32")

array([[0]], dtype=int32)